In [45]:
import geopandas as gpd
import folium

# Load the GeoJSON file
geojson_path = "~/Downloads/London2.geojson"  # Replace with your file path
gdf = gpd.read_file(geojson_path)

# Check the first few rows and columns of the GeoDataFrame
print(gdf.head())
print(gdf.columns)

# List of location codes to check
locations = [
    'E09000019', 'E09000004', 'E09000025', 'E09000011', 'E09000003', 'E09000007',
    'E09000005', 'E06000039', 'E09000006', 'E09000018', 'E09000030', 'E09000010',
    'E09000029', 'E09000023', 'E09000009', 'E09000022', 'E09000033', 'E09000026',
    'E09000014', 'E09000002', 'E09000032', 'E09000008', 'E09000015', 'E09000012',
    'E09000028', 'E09000031', 'E09000021', 'E09000013', 'E09000024', 'E09000027',
    'E09000017', 'E09000016', 'E07000215', 'E09000001', 'E07000111', 'E07000102',
    'E09000020', 'E07000068', 'E06000034', 'E07000072', 'E07000211', 'E07000095',
    'E07000006', 'E07000207', 'E06000040'
]
print(len(locations))
# E07000006 -> "E06000060", South Bucks → Buckinghamshire Council as of 2020
# this take 2013 boundaries
# Get the set of LAD13CD codes from the GeoDataFrame
geojson_codes = set(gdf["LAD13CD"].unique())

# Get the set of codes from your locations array
location_codes = set(locations)

# Identify the missing code(s)
missing_codes = location_codes - geojson_codes

# Output the result
if missing_codes:
    print("The following LAD13CD codes are in your locations array but not in the GeoJSON file:")
    print(missing_codes)
else:
    print("All LAD13CD codes are present in the GeoJSON file.")
# Filter the GeoDataFrame to only include the specified LAD13CD codes
gdf = gdf[gdf["LAD13CD"].isin(locations)]

# Check the size and structure of the filtered GeoDataFrame
num_rows, num_columns = gdf.shape
print(f"Filtered GeoDataFrame Size: {num_rows} rows, {num_columns} columns")

# Count unique boroughs in the filtered dataset
unique_boroughs = gdf["LAD13NM"].nunique()
print(f"Unique Boroughs in Filtered Data: {unique_boroughs}")

# Initialize the map centered on London
m = folium.Map(location=[51.5074, -0.1278], zoom_start=9, tiles="cartodb dark_matter")

# Sample data for visualization using only the filtered locations
data = pd.DataFrame({
    "Borough Code": gdf["LAD13CD"].unique(),
    "Value": np.random.randint(1, 100, size=len(gdf["LAD13CD"].unique()))  # Random data for visualization
})

# Add a choropleth layer using the LAD13CD column
Choropleth(
    geo_data=gdf,
    name="choropleth",
    data=data,
    columns=["Borough Code", "Value"],
    key_on="feature.properties.LAD13CD",
    fill_color="OrRd",  # Intense Orange-Red color map
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name="Selected London Borough Data",
    nan_fill_color="gray",
    highlight=True
).add_to(m)

# Add tooltips to display borough name and code
for _, row in gdf.iterrows():
    if row['geometry'] is not None:
        folium.GeoJson(
            row['geometry'],
            style_function=lambda x: {'fillColor': 'transparent', 'color': 'orange', 'weight': 2},
            tooltip=Tooltip(f"<strong>Borough:</strong> {row['LAD13NM']}<br><strong>Code:</strong> {row['LAD13CD']}")
        ).add_to(m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

# Display the map directly in the notebook
m

     LAD13CD LAD13CDO               LAD13NM LAD13NMW  \
0  E06000001     00EB            Hartlepool     None   
1  E06000002     00EC         Middlesbrough     None   
2  E06000003     00EE  Redcar and Cleveland     None   
3  E06000004     00EF      Stockton-on-Tees     None   
4  E06000005     00EH            Darlington     None   

                                            geometry  
0  MULTIPOLYGON (((-1.24196 54.72245, -1.24251 54...  
1  MULTIPOLYGON (((-1.19705 54.58178, -1.19436 54...  
2  MULTIPOLYGON (((-1.20099 54.57759, -1.20024 54...  
3  MULTIPOLYGON (((-1.30387 54.56514, -1.30199 54...  
4  POLYGON ((-1.63799 54.61719, -1.63766 54.61669...  
Index(['LAD13CD', 'LAD13CDO', 'LAD13NM', 'LAD13NMW', 'geometry'], dtype='object')
45
All LAD13CD codes are present in the GeoJSON file.
Filtered GeoDataFrame Size: 45 rows, 5 columns
Unique Boroughs in Filtered Data: 45
